In [38]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
from nltk import WordNetLemmatizer 
from nltk.corpus import wordnet 
from nltk import pos_tag 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay,RocCurveDisplay,roc_auc_score
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

#### Load Data

In [39]:
df = pd.read_csv('Tweet_Product_Company.csv',encoding = 'ISO-8859-1')
df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


#### Rename Columns

In [40]:
df = df.rename(columns={"is_there_an_emotion_directed_at_a_brand_or_product":"emotion",
                   "emotion_in_tweet_is_directed_at":"emotion app","tweet_text":"tweet text"})
df

,tweet text,emotion app,emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweet text   9092 non-null   object
 1   emotion app  3291 non-null   object
 2   emotion      9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


#### Find NAs

In [42]:
df.isna().sum()

tweet text        1
emotion app    5802
emotion           0
dtype: int64

#### Drop Columns

In [43]:
df = df.drop('emotion app', axis=1)
df

,tweet text,emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion
...,...,...
9088,Ipad everywhere. #SXSW {link},Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,No emotion toward brand or product


In [44]:
df["emotion"].unique()

array(['Negative emotion', 'Positive emotion',
       'No emotion toward brand or product', "I can't tell"], dtype=object)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet text  9092 non-null   object
 1   emotion     9093 non-null   object
dtypes: object(2)
memory usage: 142.2+ KB


#### Change Column Type to Integer

In [46]:
df.emotion.value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: emotion, dtype: int64

In [47]:
df_p = df[df.emotion == 'Positive emotion']

In [48]:
df_n = df[df.emotion == 'Negative emotion']

In [49]:
df_n.emotion = 0

/var/folders/bm/m4lrvb8117nc1ks_2lybh8hc0000gn/T/ipykernel_552/271213011.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_n.emotion = 0


In [50]:
df_n

,tweet text,emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,0
3,@sxsw I hope this year's festival isn't as cra...,0
17,I just noticed DST is coming this weekend. How...,0
38,@mention - False Alarm: Google Circles Not Co...,0
64,Again? RT @mention Line at the Apple store is ...,0
...,...,...
8973,Google guy at #sxsw talk is explaining how he ...,0
8981,I think my effing hubby is in line for an #iPa...,0
9008,I'm pretty sure the panelist that thinks &quot...,0
9043,Hey is anyone doing #sxsw signing up for the g...,0


In [51]:
df_p.emotion =1
df_p

/var/folders/bm/m4lrvb8117nc1ks_2lybh8hc0000gn/T/ipykernel_552/1114526098.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_p.emotion =1


,tweet text,emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,1
2,@swonderlin Can not wait for #iPad 2 also. The...,1
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,1
7,"#SXSW is just starting, #CTIA is around the co...",1
8,Beautifully smart and simple idea RT @madebyma...,1
...,...,...
9072,@mention your iPhone 4 cases are Rad and Ready...,1
9077,@mention your PR guy just convinced me to swit...,1
9079,&quot;papyrus...sort of like the ipad&quot; - ...,1
9085,I've always used Camera+ for my iPhone b/c it ...,1


In [52]:
df_p.head()

,tweet text,emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,1
2,@swonderlin Can not wait for #iPad 2 also. The...,1
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,1
7,"#SXSW is just starting, #CTIA is around the co...",1
8,Beautifully smart and simple idea RT @madebyma...,1


In [53]:
df_n.head()

,tweet text,emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,0
3,@sxsw I hope this year's festival isn't as cra...,0
17,I just noticed DST is coming this weekend. How...,0
38,@mention - False Alarm: Google Circles Not Co...,0
64,Again? RT @mention Line at the Apple store is ...,0


#### Merge Dataframes

In [54]:
data = pd.concat([df_n,df_p], ignore_index = True)

In [55]:
data

,tweet text,emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,0
1,@sxsw I hope this year's festival isn't as cra...,0
2,I just noticed DST is coming this weekend. How...,0
3,@mention - False Alarm: Google Circles Not Co...,0
4,Again? RT @mention Line at the Apple store is ...,0
...,...,...
3543,@mention your iPhone 4 cases are Rad and Ready...,1
3544,@mention your PR guy just convinced me to swit...,1
3545,&quot;papyrus...sort of like the ipad&quot; - ...,1
3546,I've always used Camera+ for my iPhone b/c it ...,1


In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3548 entries, 0 to 3547
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet text  3548 non-null   object
 1   emotion     3548 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 55.6+ KB


In [58]:
data.head()
data.iloc[:,-1].value_counts()

1    2978
0     570
Name: emotion, dtype: int64

#### Balance Data

In [59]:
ones = data[data.iloc[:,-1] == 1]
zeros = data[data.iloc[:,-1] == 0]

In [60]:
ones.iloc[:570,-1].value_counts()

1    570
Name: emotion, dtype: int64

In [61]:
onesdf = ones.iloc[:570,:]


In [62]:
onesdf.shape

(570, 2)

In [63]:
zeros.shape

(570, 2)

In [64]:
data_ = pd.concat([onesdf,zeros],axis=0)
data_['emotion'].value_counts()

1    570
0    570
Name: emotion, dtype: int64

In [65]:
data['tweet text']

0       .@wesley83 I have a 3G iPhone. After 3 hrs twe...
1       @sxsw I hope this year's festival isn't as cra...
2       I just noticed DST is coming this weekend. How...
3       @mention  - False Alarm: Google Circles Not Co...
4       Again? RT @mention Line at the Apple store is ...
                              ...                        
3543    @mention your iPhone 4 cases are Rad and Ready...
3544    @mention your PR guy just convinced me to swit...
3545    &quot;papyrus...sort of like the ipad&quot; - ...
3546    I've always used Camera+ for my iPhone b/c it ...
3547                        Ipad everywhere. #SXSW {link}
Name: tweet text, Length: 3548, dtype: object

#### Vectorize Data

In [66]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data_['tweet text'])
X

<1140x3286 sparse matrix of type '<class 'numpy.int64'>'
	with 19123 stored elements in Compressed Sparse Row format>

#### Conduct a Train Test Set

In [85]:
y = data_['emotion']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
X_train

<798x3286 sparse matrix of type '<class 'numpy.int64'>'
	with 13421 stored elements in Compressed Sparse Row format>

#### Create and Train Models

In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

#### Create Models

In [69]:
models = []
model1 = LogisticRegression()
model1.fit(X_train, y_train)
models.append(model1)

In [70]:
model2 = RandomForestClassifier()
model2.fit(X_train, y_train)
models.append(model2)

In [71]:
model3 = MultinomialNB()
model3.fit(X_train, y_train)
models.append(model3)

#### Generate Classification Reports

In [72]:
for i, model in enumerate(models):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    print(f"Model {i + 1} - Accuracy: {accuracy}\nClassification Report:\n{report}\n")

Model 1 - Accuracy: 0.7690058479532164
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.80      0.76       157
           1       0.81      0.75      0.78       185

    accuracy                           0.77       342
   macro avg       0.77      0.77      0.77       342
weighted avg       0.77      0.77      0.77       342


Model 2 - Accuracy: 0.7368421052631579
Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.83      0.74       157
           1       0.82      0.65      0.73       185

    accuracy                           0.74       342
   macro avg       0.75      0.74      0.74       342
weighted avg       0.75      0.74      0.74       342


Model 3 - Accuracy: 0.7807017543859649
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.76      0.76       157
           1       0.80      0.79      0.80      

In [75]:
for model in models:
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    print(f"Model: {model}")
    print("Cross-Validation Scores:", scores)
    print(f"Mean Accuracy: {scores.mean()}\n")

Model: LogisticRegression()
Cross-Validation Scores: [0.74122807 0.75       0.70175439 0.75438596 0.71491228]
Mean Accuracy: 0.7324561403508772

Model: RandomForestClassifier()
Cross-Validation Scores: [0.72807018 0.74122807 0.67982456 0.71491228 0.69736842]
Mean Accuracy: 0.7122807017543858

Model: MultinomialNB()
Cross-Validation Scores: [0.73684211 0.72807018 0.71052632 0.73245614 0.71491228]
Mean Accuracy: 0.7245614035087719



#### NLP Preprocessing - Lemmatization

In [76]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.pyplot as plt
import string
import re
import numpy as np

In [77]:
nltk.download("stopwords")
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mattgracer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [78]:
from nltk.corpus import stopwords

# get common stop words in english that we'll remove during tokenization/text normalization
stop_words = stopwords.words('english')
print(stop_words[0:5])

['i', 'me', 'my', 'myself', 'we']


In [79]:
def process_doc(doc):

    #initialize lemmatizer
    wnl = WordNetLemmatizer()

    # helper function to change nltk's part of speech tagging to a wordnet format.
    def pos_tagger(nltk_tag):
        if nltk_tag.startswith('J'):
            return wordnet.ADJ
        elif nltk_tag.startswith('V'):
            return wordnet.VERB
        elif nltk_tag.startswith('N'):
            return wordnet.NOUN
        elif nltk_tag.startswith('R'):
            return wordnet.ADV
        else:         
            return None
        
    # remove stop words and punctuations, then lower case
    doc_norm = [tok.lower() for tok in word_tokenize(doc) if ((tok.isalpha()) & (tok not in stop_words)) ]

    #  POS detection on the result will be important in telling Wordnet's lemmatizer how to lemmatize
    
    # creates list of tuples with tokens and POS tags in wordnet format
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tag(doc_norm))) 
    doc_norm = [wnl.lemmatize(token, pos) for token, pos in wordnet_tagged if pos is not None]
    
    return doc_norm

In [80]:
print(process_doc(data['tweet text'].iloc[0]))

['i', 'iphone', 'hrs', 'tweet', 'dead', 'i', 'need', 'upgrade', 'plugin', 'station', 'sxsw']


In [81]:
fully_normalized_corpus = data['tweet text'].apply(process_doc)
fully_normalized_corpus

0       [i, iphone, hrs, tweet, dead, i, need, upgrade...
1       [sxsw, i, hope, year, festival, crashy, year, ...
2       [i, notice, dst, come, weekend, many, iphone, ...
3       [mention, false, alarm, google, circle, not, c...
4       [again, rt, mention, line, apple, store, insan...
                              ...                        
3543    [mention, iphone, case, rad, ready, stop, tomo...
3544    [mention, pr, guy, convince, switch, back, iph...
3545    [quot, papyrus, sort, ipad, quot, nice, lol, s...
3546    [i, always, use, iphone, image, stabilizer, mo...
3547                       [ipad, everywhere, sxsw, link]
Name: tweet text, Length: 3548, dtype: object

In [82]:
import itertools
flattened_fully_norm = pd.Series(list(itertools.chain(*fully_normalized_corpus)))
len(flattened_fully_norm.unique())

4629

In [83]:
flattened_fully_norm

0                 i
1            iphone
2               hrs
3             tweet
4              dead
            ...    
41471         sxswi
41472          ipad
41473    everywhere
41474          sxsw
41475          link
Length: 41476, dtype: object

In [84]:
flattened_fully_norm.value_counts()

sxsw          3681
mention       2510
link          1327
ipad          1185
rt            1074
              ... 
cmswire          1
login            1
cart             1
sharp            1
stabilizer       1
Length: 4629, dtype: int64